In [ ]:
import numpy as np
import cv2 as cv
import os
import json
import matplotlib.pyplot as plt
from PIL import Image
from lang_sam import LangSAM
from scipy.spatial.transform import Rotation
from lac.localization.ekf import EKF
import typing as T
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from lac.utils.plotting import pose_trace, vector_trace, plot_reference_frames
from lac.utils.frames import (
    invert_transform_mat,
    get_cam_pose_rover,
    get_opencv_pose_rover,
    invert_transform_mat,
)
from lac.perception.vision import FiducialLocalizer
from lac.util import load_data, skew_symmetric
from lac.utils.plotting import plot_path_3d, plot_3d_points
from lac.localization.imu_dynamics import propagate_state
# import jax.numpy as jnp
# import jax

%load_ext autoreload
%autoreload 2

# Given an image containing the lander, generate range and bearing measurements to the lander.


In [ ]:
# # Define camera intrinsics
# W, H = 1280, 720
# FOV = 1.22  # radians
# focal_length_x = W / (2 * np.tan(FOV / 2))
# focal_length_y = H / (2 * np.tan(FOV / 2))
# K = np.array([[focal_length_x, 0, W / 2], [0, focal_length_x, H / 2], [0, 0, 1]])

# # Load camera geometry relative to rover
# cams_geoms = json.load(open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json")))[
#     "rover"
# ]["cameras"]

In [ ]:
# def rover_pose_world_traj(data_log: T.Dict[str, T.Any]) -> T.Tuple[np.ndarray, np.ndarray]:
#     """Extract rover pose (4x4 matrices) trajectory from data log with corresponding timestamps"""

#     ROVER_TRAJ_KEYS = ["timestamp", "pose"]
#     rover_traj_log = {key: [] for key in ROVER_TRAJ_KEYS}

#     frame_data = data_log["frames"]
#     for frame in frame_data:
#         rover_traj_log["timestamp"].append(frame["timestamp"])
#         rover_traj_log["pose"].append(frame["pose"])

#     return np.array(rover_traj_log["pose"]), np.array(rover_traj_log["timestamp"])

In [ ]:
## ---- TEST IMAGES FOR FIDUCIAL SHAPE SEG -------------------------------
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")

# data_collection_1
# true examples
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/35.png # group D
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/124.png # group D
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/1527.png # group A
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/1620.png # group A

# false examples
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/328.png
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/619.png
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/974.png

# lander_closeups
# true examples
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/53.png

# i = [35, 124, 328, 619, 974, 1527, 1620]
# ------------------------------------------------------------


# for ind in i:
#     I1 = cv.imread(os.path.join(data_path, "FrontLeft", f"{ind}.png"), cv.IMREAD_GRAYSCALE)
#     plt.imshow(I1, cmap="gray")
#     plt.show()

# Without fiducials

Use segmentation to identify the lander outline. Next, identify extreme points on the lander outline.
Assuming that we know the lander geometry, we know the location of these points in 3D relative to the
lander center.

Alternatively, we can try to directly predict the lander center (in the image in image coordinates)
from the segmentation.


In [ ]:
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/lander_no_fiducials_left_far/")
# CAM_NAME = "Left"
# imgs = os.listdir(os.path.join(data_path, CAM_NAME))
# imgs = [os.path.join(data_path, CAM_NAME, imgs[i]) for i in range(0, len(imgs), 1)]
# imgs.sort()

# # Segment lander
# model = LangSAM()
# text_prompt = "lander."
# SEG_SCORE_THRESH = 0.4 # requires more testing

# for img in imgs:
#     I1 = cv.imread(os.path.join(data_path, CAM_NAME, img), cv.IMREAD_GRAYSCALE)
#     plt.imshow(I1, cmap="gray")
#     plt.show()

#     image_seg_in = Image.open(os.path.join(data_path, CAM_NAME, img)).convert("RGB")
#     results = model.predict([image_seg_in], [text_prompt])
#     full_mask = np.zeros_like(image_seg_in).copy()
#     if len(results) == 0:
#         print("No masks found")
#     for i, mask in enumerate(results[0]["masks"]):
#         print(results[0]["scores"][i])
#         if results[0]["scores"][i] < SEG_SCORE_THRESH:
#             continue
#         full_mask[mask.astype(bool)] = 255

#         # convert mask to pixel coordinates to get min area rect
#         mask = mask.astype(np.uint8)
#         contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#         if contours:
#             largest_contour = max(contours, key=cv.contourArea)  # Pick the biggest object
#             rect = cv.minAreaRect(largest_contour)
#             box = cv.boxPoints(rect)
#             box = np.intp(box)
#             cv.drawContours(full_mask, [box], 0, (0, 0, 255), 2)

#             # sort the box points by the y coordinate
#             box = box[np.argsort(box[:, 1])]
#             lower_corners = box[2:]
#             mid_point = np.mean(lower_corners, axis=0)
#             cv.circle(full_mask, tuple(mid_point.astype(int)), 10, (0, 0, 255), -1)

#         # ------ PRIOR ATTEMPTS TO GET CENTER POINT BETWEEN LANDER FEET ------
#         # # get bounding box
#         # x, y, w, h = cv.boundingRect(mask)
#         # cv.rectangle(full_mask, (x, y), (x+w, y+h), (0, 255, 0), 2)

#         # x_avg = int(x + w/2)
#         # y_max = int(y + h)
#         # cv.circle(full_mask, (x_avg, y_max), 10, (0, 255, 0), -1)

#         # # get average x and max y of the mask to get the center point
#         # mask_x_avg = int(np.mean(np.where(mask)[1]))
#         # mask_y_max = int(np.max(np.where(mask)[0]))
#         # cv.circle(full_mask, (mask_x_avg, mask_y_max), 10, (255, 0, 0), -1)

#     plt.imshow(full_mask, cmap="gray")
#     plt.show()


In [ ]:
# def get_roteuler_tvec_from_poselist(poses: T.Union[T.List[np.ndarray], np.ndarray]) -> tuple:
#     """
#     Get Euler angles and translation vectors from poses. 4x4 for a single pose, Nx4x4 for a list of poses.
#     Input:
#         poses - list of poses or a single pose
#     Output:
#         Returns a tuple of Euler angles and tvec (lists if input is a list of poses)
#     """

#     def pose_to_roteuler_tvec(pose: np.ndarray) -> tuple:
#         """Get rpy and tvec from a single pose"""
#         r = Rotation.from_matrix(pose[:3, :3])
#         rpy = r.as_euler("xyz", degrees=False)
#         tvec = pose[:3, 3]
#         return rpy, tvec

#     if len(poses.shape) == 3 or isinstance(poses, list):  # list of poses
#         rpy, tvec = [], []
#         for pose in poses:
#             r, t = pose_to_roteuler_tvec(pose)
#             rpy.append(r)
#             tvec.append(t)
#         return rpy, tvec

#     rpy, tvec = pose_to_roteuler_tvec(poses)
#     return rpy, tvec

# def rot_from_rpy(roll, pitch, yaw, type: str = "passive") -> np.ndarray:
#     """ ZYX (pitch-roll-yaw) rotation matrix. See Tait-Bryan angles ZYX on Wikipedia. This is the passive rotation matrix. """
#     rot_passive = np.array(
#         [
#             [np.cos(yaw)*np.cos(pitch), np.cos(pitch)*np.sin(yaw), -np.sin(pitch)],
#             [np.cos(yaw)*np.sin(roll)*np.sin(pitch)-np.cos(roll)*np.sin(yaw), np.cos(roll)*np.cos(yaw)+np.sin(roll)*np.sin(yaw)*np.sin(pitch), np.cos(pitch)*np.sin(roll)],
#             [np.sin(roll)*np.sin(yaw)+np.cos(roll)*np.cos(yaw)*np.sin(pitch), np.cos(roll)*np.sin(yaw)*np.sin(pitch)-np.cos(yaw)*np.sin(roll), np.cos(roll)*np.cos(pitch)]
#         ]
#     )
#     if type == "active":
#         return rot_passive.T
#     return rot_passive

# # def drot_from_rpy(roll, pitch, yaw, type: str = "passive") -> np.ndarray:
# #     """ Derivative of ZYX (pitch-roll-yaw) rotation matrix w.r.t. rpy. """
# #     drot_dr = np.zeros((3, 3))
# #     drot_dr[1, 0] = np.cos(yaw) * np.cos(roll) * np.sin(pitch) + np.sin(roll) * np.sin(yaw)
# #     drot_dr[1, 1] = -np.sin(roll) * np.cos(yaw) + np.cos(roll) * np.sin(yaw) * np.sin(pitch)
# #     drot_dr[1, 2] = np.cos(pitch) * np.cos(roll)
# #     drot_dr[2, 0] = np.cos(roll)*np.sin(yaw) - np.sin(roll)*np.cos(yaw)*np.sin(pitch)
# #     drot_dr[2, 1] = -np.sin(roll)*np.sin(yaw)*np.sin(pitch) - np.cos(roll)*np.cos(yaw)
# #     drot_dr[2, 2] = -np.sin(roll)*np.cos(pitch)


# #     drot_dp = np.zeros((3, 3))
# #     drot_dp[0, 0] = -np.cos(yaw)*np.sin(pitch)
# #     drot_dp[0, 1] = -np.sin(yaw)*np.sin(pitch)
# #     drot_dp[0, 2] = -np.cos(pitch)
# #     drot_dp[1, 0] = np.cos(yaw)*np.cos(pitch)*np.sin(roll)
# #     drot_dp[1, 1] = np.sin(roll)*np.sin(yaw)*np.cos(pitch)
# #     drot_dp[1, 2] = -np.sin(pitch)*np.sin(roll)
# #     drot_dp[2, 0] = np.cos(roll)*np.cos(yaw)*np.cos(pitch)
# #     drot_dp[2, 1] = np.cos(roll)*np.sin(yaw)*np.cos(pitch)
# #     drot_dp[2, 2] = -np.cos(roll)*np.sin(pitch)

# #     drot_dy = np.zeros((3, 3))
# #     drot_dy[0, 0] = -np.sin(yaw)*np.cos(pitch)
# #     drot_dy[0, 1] = np.cos(yaw)*np.cos(pitch)
# #     drot_dy[1, 0] = -np.sin(yaw)*np.sin(roll)*np.sin(pitch) - np.cos(roll)*np.cos(yaw)
# #     drot_dy[1, 1] = -np.cos(roll)*np.sin(yaw) + np.cos(yaw)*np.sin(roll)*np.sin(pitch)
# #     drot_dy[2, 0] = np.sin(roll)*np.cos(yaw) - np.cos(roll)*np.sin(yaw)*np.sin(pitch)
# #     drot_dy[2, 1] = np.cos(roll)*np.cos(yaw)*np.sin(pitch) + np.sin(roll)*np.sin(yaw)

# #     if type == "active":
# #         return drot_dr.T, drot_dp.T, drot_dy.T
# #     return drot_dr, drot_dp, drot_dy


# class MeasModelLanderSeg:
#     def __init__(
#         self,
#         model,
#         lander_pose: np.ndarray,
#         cam_name: str,
#         cam_geoms: dict,
#         seg_thresh: float = 0.4,
#         K: np.ndarray = K,
#         R_nom: np.ndarray = np.eye(2) * 1,

#     ):
#         # segmentation
#         self.model = model
#         self.seg_thresh = seg_thresh

#         # known parameters
#         self.cam_name = cam_name
#         self.cam_pose_rover = get_cam_pose_rover(cam_name) # camera pose w.r.t. rover
#         self.opencv_pose_rover = get_opencv_pose_rover(cam_name, cam_geoms) # opencv camera pose w.r.t. rover
#         self.lander_pose_world = lander_pose # lander pose w.r.t. world
#         self.K = K  # camera intrinsics

#         # measurement noise
#         self.R = R_nom  # measurement noise covariance matrix under nominal conditions

#     def rover_pose_from_statearr(self, state_arr: np.ndarray) -> np.ndarray:
#         """Get the rover pose w.r.t. world from the state array"""
#         rover_pos = state_arr[:3]
#         rover_rpy = state_arr[6:]
#         rover_rot = Rotation.from_euler("xyz", rover_rpy, degrees=False).as_matrix()
#         rover_pose = np.eye(4)
#         rover_pose[:3, :3] = rover_rot
#         rover_pose[:3, 3] = rover_pos
#         return rover_pose

#     def get_lander_origin_pix_from_image(self, img_path: str, display: bool = False) -> np.ndarray:
#         """Measurement function for extracting the lander origin pixel coordinates from an image"""

#         lander_origin = None

#         # Segment lander
#         image_seg_in = Image.open(img_path).convert("RGB")
#         text_prompt = "lander."
#         results = self.model.predict([image_seg_in], [text_prompt])
#         full_mask = np.zeros_like(image_seg_in).copy()

#         if len(results) == 0:  # no masks found
#             print("No masks found")
#             return lander_origin

#         for i, mask in enumerate(results[0]["masks"]):
#             if results[0]["scores"][i] < self.seg_thresh:
#                 continue

#             full_mask[mask.astype(bool)] = 255
#             mask = mask.astype(np.uint8)
#             contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

#             if contours:
#                 largest_contour = max(contours, key=cv.contourArea)
#                 rect = cv.minAreaRect(largest_contour)
#                 box = cv.boxPoints(rect)
#                 box = np.intp(box)

#                 # sort the box points by the y coordinate to get the lower corners (lander feet)
#                 sorted_box = box[np.argsort(box[:, 1])]
#                 lower_corners = sorted_box[2:]
#                 mid_point = np.mean(lower_corners, axis=0)
#                 lander_origin = mid_point

#                 if display:
#                     cv.drawContours(full_mask, [box], 0, (0, 0, 255), 2)
#                     cv.circle(full_mask, tuple(mid_point.astype(int)), 10, (0, 0, 255), -1)

#         if display:
#             plt.imshow(full_mask, cmap="gray")
#             plt.show()

#         return lander_origin

#     def get_lander_origin_pix_from_proj(
#         self, rover_state: np.ndarray, display: bool = False
#     ) -> np.ndarray:
#         """
#         Compute the lander origin pixel coordinates via projection to the image plane.
#         NOTE: X_rot_Y is the rotation matrix that rotates the Y frame to the X frame.
#         NOTE: rpy is roll, pitch, yaw in radians.

#         Input:
#             rover_state - state of the rover w.r.t. the world frame
#         Output:
#             lander_origin_pix - lander origin pixel coordinates [u, v]
#             pix_coords - unormalized pixel coordinates [u, v, z]
#         """

#         rover_pose_world = self.rover_pose_from_statearr(rover_state)
#         cam_pose_world = rover_pose_world @ self.cam_pose_rover
#         opencv_pose_world = rover_pose_world @ self.opencv_pose_rover

#         pix_coords = self.K @ np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]]) @ invert_transform_mat(opencv_pose_world) @ np.hstack((self.lander_pose_world[:3, 3], [1])) # inverse transform to get passive transformation

#         # TODO: so maybe the reporjection is using active? well either the plotting or the repojection is using active and the other is passive.
#         # opencv_transform_inv = invert_transform_mat(opencv_pose_world)
#         # print(opencv_transform_inv)
#         # inv_angs = Rotation.from_matrix(opencv_transform_inv[:3, :3]).as_euler("ZYX", degrees=False)
#         # print(rot_from_rpy(*inv_angs[::-1], type="active"))

#         if np.all(pix_coords[2] > 0):  # Check if the point is in front of the camera
#             lander_origin_pix = pix_coords[:2] / pix_coords[2]
#         else:
#             print("Lander is behind the camera.")
#             return None, None

#         if display:
#             world_pose = np.eye(4)

#             fig_world = plot_reference_frames([world_pose, self.lander_pose_world, rover_pose_world, cam_pose_world, opencv_pose_world], ["world", "lander", "rover", "cam", "opencv"])

#             rover_pos_world_vectrace = vector_trace(
#                 np.zeros(3),
#                 rover_pose_world[:3, 3],
#                 color="blue",
#                 name="[k] rover_pos w.r.t. world in world-frame",
#             )
#             lander_pos_world_vectrace = vector_trace(
#                 np.zeros(3),
#                 self.lander_pose_world[:3, 3],
#                 color="purple",
#                 name="[k] lander_pos w.r.t. world in world-frame",
#             )
#             cam_pos_world_vectrace = vector_trace(
#                 np.zeros(3), cam_pose_world[:3, 3], color="green", name="[c] cam_pos w.r.t. world in world-frame"
#             )

#             cam_pos_rover_rover = self.cam_pose_rover[:3, 3]
#             cam_pos_rover_world = rover_pose_world[:3, :3] @ cam_pos_rover_rover
#             cam_pos_rover_world_vectrace = vector_trace(
#                 rover_pose_world[:3, 3],
#                 rover_pose_world[:3, 3] + cam_pos_rover_world,
#                 color="orange",
#                 name="[c] cam_pos w.r.t. rover in world",
#                 head_size=0.1,
#             )

#             for trace in lander_pos_world_vectrace:
#                 fig_world.add_trace(trace)
#             for trace in rover_pos_world_vectrace:
#                 fig_world.add_trace(trace)
#             for trace in cam_pos_world_vectrace:
#                 fig_world.add_trace(trace)
#             for trace in cam_pos_rover_world_vectrace:
#                 fig_world.add_trace(trace)

#             fig_world.update_layout(height=700, width=1200, scene_aspectmode="data")
#             fig_world.show()

#         return lander_origin_pix[0:2], pix_coords

#     def _dudx(self, rover_state, opencv_pose_rover, cam_pose_world):
#         Theta, Phi, Psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         xc, yc, zc = cam_pose_world[:3, 3]
#         xr, yr, zr = rover_state[:3]
#         fx = self.K[0, 0]
#         W = self.K[0, 2] * 2

#         denominator = (
#             (np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.sin(Phi) * r[2, 2]) * (X - xc - xr)
#             + ((np.cos(Psi) * np.sin(Theta) * np.sin(Phi) - np.cos(Theta) * np.sin(Psi)) * r[0, 2]
#             + (np.cos(Theta) * np.cos(Psi) + np.sin(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Phi) * np.sin(Theta) * r[2, 2]) * (Y - yc - yr)
#             + ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 2]
#             + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Theta) * np.cos(Phi) * r[2, 2]) * (Z - zc - zr)
#         )

#         numerator = (
#             -fx * (np.cos(Phi) * np.cos(Psi) * r[0, 0] + np.cos(Phi) * np.sin(Psi) * r[1, 0] - np.sin(Phi) * r[2, 0])
#             - 0.5 * W * (np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.sin(Phi) * r[2, 2])
#         )

#         return numerator / denominator


#     def _dudyaw(self, rover_state, opencv_pose_rover, cam_pose_world):
#         Theta, Phi, Psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         xc, yc, zc = cam_pose_world[:3, 3]
#         xr, yr, zr = rover_state[:3]
#         fx = self.K[0, 0]
#         W = self.K[0, 2] * 2

#         # Compute denominator
#         denominator = (
#             (np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.sin(Phi) * r[2, 2]) * (X - xc - xr)
#             + ((np.cos(Psi) * np.sin(Theta) * np.sin(Phi) - np.cos(Theta) * np.sin(Psi)) * r[0, 2]
#             + (np.cos(Theta) * np.cos(Psi) + np.sin(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Phi) * np.sin(Theta) * r[2, 2]) * (Y - yc - yr)
#             + ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 2]
#             + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Theta) * np.cos(Phi) * r[2, 2]) * (Z - zc - zr)
#         )

#         # Compute numerator
#         numerator = (
#             -fx * ((np.cos(Psi) * np.sin(Theta) * np.sin(Phi) - np.cos(Theta) * np.sin(Psi)) * r[0, 0]
#                 + (np.cos(Theta) * np.cos(Psi) + np.sin(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 0]
#                 + np.cos(Phi) * np.sin(Theta) * r[2, 0])
#             - 0.5 * W * ((np.cos(Psi) * np.sin(Theta) * np.sin(Phi) - np.cos(Theta) * np.sin(Psi)) * r[0, 2]
#                         + (np.cos(Theta) * np.cos(Psi) + np.sin(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#                         + np.cos(Phi) * np.sin(Theta) * r[2, 2])
#         )

#         return numerator / denominator

#     def _dudz(self, rover_state, opencv_pose_rover, cam_pose_world):
#         Theta, Phi, Psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         xc, yc, zc = cam_pose_world[:3, 3]
#         xr, yr, zr = rover_state[:3]
#         fx = self.K[0, 0]
#         W = self.K[0, 2] * 2

#         # Compute denominator
#         denominator = (
#             (np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.sin(Phi) * r[2, 2]) * (X - xc - xr)
#             + ((np.cos(Psi) * np.sin(Theta) * np.sin(Phi) - np.cos(Theta) * np.sin(Psi)) * r[0, 2]
#             + (np.cos(Theta) * np.cos(Psi) + np.sin(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Phi) * np.sin(Theta) * r[2, 2]) * (Y - yc - yr)
#             + ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 2]
#             + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Theta) * np.cos(Phi) * r[2, 2]) * (Z - zc - zr)
#         )

#         # Compute numerator
#         numerator = (
#             -fx * ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 0]
#                 + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 0]
#                 + np.cos(Theta) * np.cos(Phi) * r[2, 0])
#             - 0.5 * W * ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 2]
#                         + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#                         + np.cos(Theta) * np.cos(Phi) * r[2, 2])
#         )

#         return numerator / denominator

#     def _dudroll(self, rover_state, opencv_pose_rover, cam_pose_world):
#         Theta, Phi, Psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         xc, yc, zc = cam_pose_world[:3, 3]
#         xr, yr, zr = rover_state[:3]
#         fx = self.K[0, 0]
#         W = self.K[0, 2] * 2

#         # Compute denominator
#         denominator = (
#             (np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.sin(Phi) * r[2, 2]) * (X - xc - xr)
#             + ((np.cos(Psi) * np.sin(Theta) * np.sin(Phi) - np.cos(Theta) * np.sin(Psi)) * r[0, 2]
#             + (np.cos(Theta) * np.cos(Psi) + np.sin(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Phi) * np.sin(Theta) * r[2, 2]) * (Y - yc - yr)
#             + ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 2]
#             + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Theta) * np.cos(Phi) * r[2, 2]) * (Z - zc - zr)
#         )

#         # Compute numerator
#         numerator = (
#             -fx * ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 0]
#                 + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 0]
#                 + np.cos(Theta) * np.cos(Phi) * r[2, 0])
#             - 0.5 * W * ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 2]
#                         + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#                         + np.cos(Theta) * np.cos(Phi) * r[2, 2])
#         )

#         return numerator / denominator

#     def _dudpitch(self, rover_state, opencv_pose_rover, cam_pose_world):
#         Theta, Phi, Psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         xc, yc, zc = cam_pose_world[:3, 3]
#         xr, yr, zr = rover_state[:3]
#         fx = self.K[0, 0]
#         W = self.K[0, 2] * 2

#         # Compute denominator
#         denominator = (
#             (np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.sin(Phi) * r[2, 2]) * (X - xc - xr)
#             + ((np.cos(Psi) * np.sin(Theta) * np.sin(Phi) - np.cos(Theta) * np.sin(Psi)) * r[0, 2]
#             + (np.cos(Theta) * np.cos(Psi) + np.sin(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Phi) * np.sin(Theta) * r[2, 2]) * (Y - yc - yr)
#             + ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 2]
#             + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Theta) * np.cos(Phi) * r[2, 2]) * (Z - zc - zr)
#         ) ** 2

#         # Compute numerator
#         numerator = (
#             (-fx * (-np.cos(Psi) * np.sin(Phi) * r[0, 0] - np.sin(Phi) * np.sin(Psi) * r[1, 0] - np.cos(Phi) * r[2, 0])
#             - 0.5 * W * (-np.cos(Psi) * np.sin(Phi) * r[0, 2] - np.sin(Phi) * np.sin(Psi) * r[1, 2] - np.cos(Phi) * r[2, 2]))
#             * (X - xc - xr)
#             + (-fx * (np.cos(Phi) * np.cos(Psi) * np.sin(Theta) * r[0, 0] + np.cos(Phi) * np.sin(Theta) * np.sin(Psi) * r[1, 0] - np.sin(Theta) * np.sin(Phi) * r[2, 0])
#             - 0.5 * W * (np.cos(Phi) * np.cos(Psi) * np.sin(Theta) * r[0, 2] + np.cos(Phi) * np.sin(Theta) * np.sin(Psi) * r[1, 2] - np.sin(Theta) * np.sin(Phi) * r[2, 2]))
#             * (Y - yc - yr)
#             + (-fx * (np.cos(Theta) * np.cos(Phi) * np.cos(Psi) * r[0, 0] + np.cos(Theta) * np.cos(Phi) * np.sin(Psi) * r[1, 0] - np.cos(Theta) * np.sin(Phi) * r[2, 0])
#             - 0.5 * W * (np.cos(Theta) * np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Theta) * np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.cos(Theta) * np.sin(Phi) * r[2, 2]))
#             * (Z - zc - zr)
#         )

#         return numerator / denominator

#     def _dudyaw(self, rover_state, opencv_pose_rover, cam_pose_world):
#         Theta, Phi, Psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         xc, yc, zc = cam_pose_world[:3, 3]
#         xr, yr, zr = rover_state[:3]
#         fx = self.K[0, 0]
#         W = self.K[0, 2] * 2

#         # Compute denominator
#         denominator = (
#             (np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.sin(Phi) * r[2, 2]) * (X - xc - xr)
#             + ((np.cos(Psi) * np.sin(Theta) * np.sin(Phi) - np.cos(Theta) * np.sin(Psi)) * r[0, 2]
#             + (np.cos(Theta) * np.cos(Psi) + np.sin(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Phi) * np.sin(Theta) * r[2, 2]) * (Y - yc - yr)
#             + ((np.cos(Theta) * np.cos(Psi) * np.sin(Phi) + np.sin(Theta) * np.sin(Psi)) * r[0, 2]
#             + (-np.cos(Psi) * np.sin(Theta) + np.cos(Theta) * np.sin(Phi) * np.sin(Psi)) * r[1, 2]
#             + np.cos(Theta) * np.cos(Phi) * r[2, 2]) * (Z - zc - zr)
#         ) ** 2

#         # Compute numerator
#         numerator = (
#             (-fx * (-np.cos(Psi) * np.sin(Phi) * r[0, 0] - np.sin(Phi) * np.sin(Psi) * r[1, 0] - np.cos(Phi) * r[2, 0])
#             - 0.5 * W * (-np.cos(Psi) * np.sin(Phi) * r[0, 2] - np.sin(Phi) * np.sin(Psi) * r[1, 2] - np.cos(Phi) * r[2, 2]))
#             * (X - xc - xr)
#             + (-fx * (np.cos(Phi) * np.cos(Psi) * np.sin(Theta) * r[0, 0] + np.cos(Phi) * np.sin(Theta) * np.sin(Psi) * r[1, 0] - np.sin(Theta) * np.sin(Phi) * r[2, 0])
#             - 0.5 * W * (np.cos(Phi) * np.cos(Psi) * np.sin(Theta) * r[0, 2] + np.cos(Phi) * np.sin(Theta) * np.sin(Psi) * r[1, 2] - np.sin(Theta) * np.sin(Phi) * r[2, 2]))
#             * (Y - yc - yr)
#             + (-fx * (np.cos(Theta) * np.cos(Phi) * np.cos(Psi) * r[0, 0] + np.cos(Theta) * np.cos(Phi) * np.sin(Psi) * r[1, 0] - np.cos(Theta) * np.sin(Phi) * r[2, 0])
#             - 0.5 * W * (np.cos(Theta) * np.cos(Phi) * np.cos(Psi) * r[0, 2] + np.cos(Theta) * np.cos(Phi) * np.sin(Psi) * r[1, 2] - np.cos(Theta) * np.sin(Phi) * r[2, 2]))
#             * (Z - zc - zr)
#         )

#         return numerator / denominator

#     def _dvdx(self, rover_state, opencv_pose_rover, cam_pose_world):
#         theta, phi, psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         x_c, y_c, z_c = cam_pose_world[:3, 3]
#         x_r, y_r, z_r = rover_state[:3]
#         f_x = self.K[0, 0]
#         H = self.K[1, 2] * 2

#         # Numerator
#         numerator = (-f_x * (np.cos(phi) * np.cos(psi) * r[0, 1] + np.cos(phi) * np.sin(psi) * r[1,1] - np.sin(phi) * r[2,1]) - 0.5 * H * (np.cos(phi) * np.cos(psi) * r[0,2] + np.cos(phi) * np.sin(psi) * r[1,2] - np.sin(phi) * r[2,2]))

#         # Denominator part 1
#         denom_part1 = (np.cos(phi) * np.cos(psi) * r[0,2] + np.cos(phi) * np.sin(psi) * r[1,2] - np.sin(phi) * r[2,2])

#         # Denominator part 2
#         denom_part2 = ((np.cos(psi) * np.sin(theta) * np.sin(phi) - np.cos(theta) * np.sin(psi)) * r[0,2] +
#                     (np.cos(theta) * np.cos(psi) + np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1,2] +
#                     np.cos(phi) * np.sin(theta) * r[2,2])

#         # Denominator part 3
#         denom_part3 = ((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0,2] +
#                     (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1,2] +
#                     np.cos(theta) * np.cos(phi) * r[2,2])

#         # Denominator
#         denominator = (denom_part1 * (X - x_c - x_r) +
#                     denom_part2 * (Y - y_c - y_r) +
#                     denom_part3 * (Z - z_c - z_r))

#         # Final result
#         result = numerator / denominator ** 2


#         return result


#     def _dvdy(self, rover_state, opencv_pose_rover, cam_pose_world):
#         theta, phi, psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         x_c, y_c, z_c = cam_pose_world[:3, 3]
#         x_r, y_r, z_r = rover_state[:3]
#         f_x = self.K[0, 0]
#         H = self.K[1, 2] * 2

#         # First term in the numerator
#         numerator_term1 = (-f_x * ((np.cos(psi) * np.sin(theta) * np.sin(phi) - np.cos(theta) * np.sin(psi)) * r[0, 1] + (np.cos(theta) * np.cos(psi) + np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1, 1] + np.cos(phi) * np.sin(theta) * r[2, 1]))

#         # Second term in the numerator (with H)
#         numerator_term2 = (-0.5 * H * ((np.cos(psi) * np.sin(theta) * np.sin(phi) - np.cos(theta) * np.sin(psi)) * r[0, 2] + (np.cos(theta) * np.cos(psi) + np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] + np.cos(phi) * np.sin(theta) * r[2,2]))

#         # Denominator part 1
#         denom_part1 = (np.cos(phi) * np.cos(psi) * r[0, 2] + np.cos(phi) * np.sin(psi) * r[1, 2] - np.sin(phi) * r[2, 2])

#         # Denominator part 2
#         denom_part2 = ((np.cos(psi) * np.sin(theta) * np.sin(phi) - np.cos(theta) * np.sin(psi)) * r[0, 2] +
#                     (np.cos(theta) * np.cos(psi) + np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] +
#                     np.cos(phi) * np.sin(theta) * r[2, 2])

#         # Denominator part 3
#         denom_part3 = ((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0, 2] +
#                     (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] +
#                     np.cos(theta) * np.cos(phi) * r[2, 2])

#         # Denominator
#         denominator = (denom_part1 * (X - x_c - x_r) +
#                     denom_part2 * (Y - y_c - y_r) +
#                     denom_part3 * (Z - z_c - z_r))

#         # Now we compute the full result (numerator over denominator squared)
#         result = (numerator_term1 + numerator_term2) / denominator ** 2

#         return result

#     def _dvdz(self, rover_state, opencv_pose_rover, cam_pose_world):
#         theta, phi, psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         x_c, y_c, z_c = cam_pose_world[:3, 3]
#         x_r, y_r, z_r = rover_state[:3]
#         f_x = self.K[0, 0]
#         H = self.K[1, 2] * 2


#         # First part of the numerator
#         numerator_term1 = (-f_x * ((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0, 1] + (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1, 1] + np.cos(theta) * np.cos(phi) * r[2, 1]))

#         # Second part of the numerator (with H)
#         numerator_term2 = (-0.5 * H * ((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0, 2] + (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] + np.cos(theta) * np.cos(phi) * r[2, 2]))

#         # Denominator part 1
#         denom_part1 = (np.cos(phi) * np.cos(psi) * r[0, 2] + np.cos(phi) * np.sin(psi) * r[1, 2] - np.sin(phi) * r[2, 2])

#         # Denominator part 2
#         denom_part2 = ((np.cos(psi) * np.sin(theta) * np.sin(phi) - np.cos(theta) * np.sin(psi)) * r[0, 2] +
#                     (np.cos(theta) * np.cos(psi) + np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] +
#                     np.cos(phi) * np.sin(theta) * r[2, 2])

#         # Denominator part 3
#         denom_part3 = ((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0, 2] +
#                     (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] +
#                     np.cos(theta) * np.cos(phi) * r[2, 2])

#         # Denominator
#         denominator = (denom_part1 * (X - x_c - x_r) +
#                     denom_part2 * (Y - y_c - y_r) +
#                     denom_part3 * (Z - z_c - z_r))

#         # Now we compute the full result (numerator over denominator squared)
#         result = (numerator_term1 + numerator_term2) / denominator ** 2

#     def _dvdroll(self, rover_state, opencv_pose_rover, cam_pose_world):
#         theta, phi, psi = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         x_c, y_c, z_c = cam_pose_world[:3, 3]
#         x_r, y_r, z_r = rover_state[:3]
#         f_x = self.K[0, 0]
#         H = self.K[1, 2] * 2

#         # First part of the numerator (Y term)
#         numerator_term1_Y = (((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0, 2] +
#                             (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] +
#                             np.cos(theta) * np.cos(phi) * r[2, 2]) * (Y - y_c - y_r))

#         # Second part of the numerator (Z term)
#         numerator_term2_Z = (((-np.cos(psi) * np.sin(theta) * np.sin(phi) + np.cos(theta) * np.sin(psi)) * r[0, 2] +
#                             (-np.cos(theta) * np.cos(psi) - np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] -
#                             np.cos(phi) * np.sin(theta) * r[2, 2]) * (Z - z_c - z_r))

#         # First part of the numerator (force terms for Y)
#         force_term1_Y = (f_x * (np.cos(phi) * np.cos(psi) * r[0, 1] + np.cos(phi) * np.sin(psi) * r[1,1] - np.sin(phi) * r[2, 1]) +
#                         0.5 * H * (np.cos(phi) * np.cos(psi) * r[0, 2] + np.cos(phi) * np.sin(psi) * r[1, 2] - np.sin(phi) * r[2, 2])) * (Y - y_c - y_r)

#         # Second part of the numerator (force terms for Z)
#         force_term2_Z = (f_x * ((np.cos(psi) * np.sin(theta) * np.sin(phi) - np.cos(theta) * np.sin(psi)) * r[0, 1] + (np.cos(theta) * np.cos(psi) + np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1,1] + np.cos(phi) * np.sin(theta) * r[2, 1]) + 0.5 * H * ((np.cos(psi) * np.sin(theta) * np.sin(phi) - np.cos(theta) * np.sin(psi)) * r[0, 2] + (np.cos(theta) * np.cos(psi) + np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] + np.cos(phi) * np.sin(theta) * r[2, 2])) * (Z - z_c - z_r)

#         # First part of the numerator (force terms for X)
#         force_term3_X = (f_x * ((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0, 1] + (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1,1] + np.cos(theta) * np.cos(phi) * r[2, 1]) + 0.5 * H * ((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0, 2] +  (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] + np.cos(theta) * np.cos(phi) * r[2, 2])) * (X - x_c - x_r)

#         # Denominator
#         denom_part1 = (np.cos(phi) * np.cos(psi) * r[0, 2] + np.cos(phi) * np.sin(psi) * r[1, 2] - np.sin(phi) * r[2, 2])
#         denom_part2 = ((np.cos(psi) * np.sin(theta) * np.sin(phi) - np.cos(theta) * np.sin(psi)) * r[0, 2] +
#                     (np.cos(theta) * np.cos(psi) + np.sin(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] +
#                     np.cos(phi) * np.sin(theta) * r[2, 2])
#         denom_part3 = ((np.cos(theta) * np.cos(psi) * np.sin(phi) + np.sin(theta) * np.sin(psi)) * r[0, 2] +
#                     (-np.cos(psi) * np.sin(theta) + np.cos(theta) * np.sin(phi) * np.sin(psi)) * r[1, 2] +
#                     np.cos(theta) * np.cos(phi) * r[2, 2])

#         denominator = (denom_part1 * (X - x_c - x_r) +
#                     denom_part2 * (Y - y_c - y_r) +
#                     denom_part3 * (Z - z_c - z_r))

#         # Final result
#         result = -((numerator_term1_Y + numerator_term2_Z) * (force_term1_Y + force_term2_Z + force_term3_X)) / (denominator ** 2)
#         return result

#     def _dvdpitch(self, rover_state, opencv_pose_rover, cam_pose_world):
#         roll, pitch, yaw = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         x_c, y_c, z_c = cam_pose_world[:3, 3]
#         x_r, y_r, z_r = rover_state[:3]
#         fx = self.K[0, 0]
#         H = self.K[1, 2] * 2

#         # Compute the components (example for the first part of the expression)
#         cp = np.cos(pitch)
#         sp = np.sin(pitch)
#         cy = np.cos(yaw)
#         sy = np.sin(yaw)
#         cr = np.cos(roll)
#         sr = np.sin(roll)

#         # First term in the numerator
#         term1 = (-cy * sp * r[0, 2] - sp * sy * r[1, 2] - cp * r[2, 2]) * (X - x_c - x_r)
#         term2 = (cp * cy * sr * r[0, 2] + cp * sr * sy * r[1, 2] - sr * sp * r[2, 2]) * (Y - y_c - y_r)
#         term3 = (cr * cp * cy * r[0, 2] + cr * cp * sy * r[1, 2] - cr * sp * r[2, 2]) * (Z - z_c - z_r)

#         # Sum of these terms for the first part of the numerator
#         numerator_first_part = term1 + term2 + term3

#         # Second part of the numerator (the product of f_x terms)
#         term4 = (fx * (cp * cy * r[0, 1] + cp * sy * r[1,1] - sp * r[2, 1]) +
#                 0.5 * H * (cp * cy * r[0, 2] + cp * sy * r[1, 2]- sp * r[2, 2])) * (X - x_c - x_r)

#         term5 = (fx * (cy * sr * sp - cr * sy) * r[0, 1] +
#                 (cr * cy + sr * sp * sy) * r[1,1] + cp * sr * r[2, 1]) * (Y - y_c - y_r)

#         # Similarly for the Z component of the second part
#         term6 = (fx * (cr * cy * sp + sr * sy) * r[0, 1] +
#                 (-cy * sr + cr * sp * sy) * r[1,1] + cr * cp * r[2, 1]) * (Z - z_c - z_r)

#         # Sum the second part of the numerator
#         numerator_second_part = term4 + term5 + term6

#         # Now handle the denominator
#         denom_term1 = (cp * cy * r[0, 2] + cp * sy * r[1, 2] - sp * r[2, 2]) * (X - x_c - x_r)
#         denom_term2 = (cy * sr * sp - cr * sy) * r[0, 2] + \
#                       (cr * cy + sr * sp * sy) * r[1, 2] + \
#                        cp * sr * r[2, 2]* (Y - y_c - y_r)
#         denom_term3 = (cr * cy * sp + sr * sy) * r[0, 2] + (-cy * sr + cr * sp * sy) * r[1, 2] + cr * cp * r[2, 2] * (Z - z_c - z_r)

#         # Denominator sum
#         denominator = denom_term1 + denom_term2 + denom_term3

#         # Now calculate the final result
#         result = (numerator_first_part + numerator_second_part) / denominator

#         return result

#     def _dvdyaw(self, rover_state, opencv_pose_rover, cam_pose_world):
#         roll, pitch, yaw = rover_state[6:]
#         X, Y, Z = self.lander_pose_world[:3, 3]
#         r = opencv_pose_rover[:3, :3]
#         x_c, y_c, z_c = cam_pose_world[:3, 3]
#         x_r, y_r, z_r = rover_state[:3]
#         fx = self.K[0, 0]
#         H = self.K[1, 2] * 2

#         # Trigonometric functions for reuse
#         cp = np.cos(pitch)
#         sp = np.sin(pitch)
#         cr = np.cos(roll)
#         sr = np.sin(roll)
#         cy = np.cos(yaw)
#         sy = np.sin(yaw)

#         # Define the first part of the numerator
#         term1 = fx * (-cp * sy * r[0, 1] + cp * cy * r[1,1]) + \
#                 0.5 * H * (-cp * sy * r[0, 2] + cp * cy * r[1, 2])

#         term2 = fx * (-cr * cy - sr * sp * sy) * r[0, 1] + \
#                 fx * (cy * sr * sp - cr * sy) * r[1,1] + \
#                 0.5 * H * (-cr * cy - sr * sp * sy) * r[0, 2] + \
#                 0.5 * H * (cy * sr * sp - cr * sy) * r[1, 2]

#         term3 = fx * (cy * sr - cr * sp * sy) * r[0, 1] + \
#                 fx * (cr * cy + sr * sp * sy) * r[1,1] + \
#                 0.5 * H * (cy * sr - cr * sp * sy) * r[0, 2] + \
#                 0.5 * H * (cr * cy + sr * sp * sy) * r[1, 2]

#         # Multiply each term with the respective coordinate difference
#         numerator_first_part = (term1 + term2 + term3) * (X - x_c - x_r)

#         # Repeat similar operations for Y and Z (for the Y and Z components of the numerator)
#         # Y component of the numerator
#         term4 = fx * (-cp * sy * r[0, 1] + cp * cy * r[1,1]) + \
#                 0.5 * H * (-cp * sy * r[0, 2] + cp * cy * r[1, 2])

#         term5 = fx * (-cr * cy - sr * sp * sy) * r[0, 1] + \
#                 fx * (cy * sr * sp - cr * sy) * r[1,1] + \
#                 0.5 * H * (-cr * cy - sr * sp * sy) * r[0, 2] + \
#                 0.5 * H * (cy * sr * sp - cr * sy) * r[1, 2]

#         term6 = fx * (cy * sr - cr * sp * sy) * r[0, 1] + \
#                 fx * (cr * cy + sr * sp * sy) * r[1,1] + \
#                 0.5 * H * (cy * sr - cr * sp * sy) * r[0, 2] + \
#                 0.5 * H * (cr * cy + sr * sp * sy) * r[1, 2]

#         numerator_second_part = (term4 + term5 + term6) * (Y - y_c - y_r)

#         # Z component of the numerator
#         term7 = fx * (-cp * sy * r[0, 1] + cp * cy * r[1,1]) + \
#                 0.5 * H * (-cp * sy * r[0, 2] + cp * cy * r[1, 2])

#         term8 = fx * (-cr * cy - sr * sp * sy) * r[0, 1] + \
#                 fx * (cy * sr * sp - cr * sy) * r[1,1] + \
#                 0.5 * H * (-cr * cy - sr * sp * sy) * r[0, 2] + \
#                 0.5 * H * (cy * sr * sp - cr * sy) * r[1, 2]

#         term9 = fx * (cy * sr - cr * sp * sy) * r[0, 1] + \
#                 fx * (cr * cy + sr * sp * sy) * r[1,1]+ \
#                 0.5 * H * (cy * sr - cr * sp * sy) * r[0, 2] + \
#                 0.5 * H * (cr * cy + sr * sp * sy) * r[1, 2]

#         numerator_third_part = (term7 + term8 + term9) * (Z - z_c - z_r)

#         # Sum of all the numerator parts
#         numerator = numerator_first_part + numerator_second_part + numerator_third_part

#         # Denominator calculation (split into three parts)
#         denom_term1 = (cp * cy * r[0, 2] + cp * sy * r[1, 2] - sp * r[2, 2]) * (X - x_c - x_r)
#         denom_term2 = (cy * sr * sp - cr * sy) * r[0, 2] + \
#                     (cr * cy + sr * sp * sy) * r[1, 2] + cp * sr * r[2, 2] * (Y - y_c - y_r)
#         denom_term3 = (cr * cy * sp + sr * sy) * r[0, 2] + \
#                     (-cy * sr + cr * sp * sy) * r[1, 2] + cr * cp * r[2, 2] * (Z - z_c - z_r)

#         # Denominator sum
#         denominator = denom_term1 + denom_term2 + denom_term3

#         # Final result
#         result = numerator / denominator
#         return result


#     def jac_reproj(self, rover_state: np.ndarray, pix_unnorm: np.ndarray) -> np.ndarray:
#         """ Jacobian matrix of the reprojection function """

#         r, p, y = rover_state[6:]

#         # get the camera pose w.r.t. the world frame
#         rover_pose_world = self.rover_pose_from_statearr(rover_state)
#         cam_pose_world = rover_pose_world @ self.cam_pose_rover
#         opencv_pose_world = rover_pose_world @ self.opencv_pose_rover

#         C = self.K @ opencv_pose_world[:3, :3].T

#         H = np.zeros((2,len(rover_state)))
#         H[0, 0] = self._dudx(rover_state, opencv_pose_world, cam_pose_world)
#         H[0, 1] = self._dudyaw(rover_state, opencv_pose_world, cam_pose_world)
#         H[0, 2] = self._dudz(rover_state, opencv_pose_world, cam_pose_world)
#         H[0, 6] = self._dudroll(rover_state, opencv_pose_world, cam_pose_world)
#         H[0, 7] = self._dudpitch(rover_state, opencv_pose_world, cam_pose_world)
#         H[0, 8] = self._dudyaw(rover_state, opencv_pose_world, cam_pose_world)
#         H[1, 0] = self._dvdx(rover_state, opencv_pose_world, cam_pose_world)
#         H[1, 1] = self._dvdy(rover_state, opencv_pose_world, cam_pose_world)
#         H[1, 2] = self._dvdz(rover_state, opencv_pose_world, cam_pose_world)
#         H[1, 6] = self._dvdroll(rover_state, opencv_pose_world, cam_pose_world)
#         H[1, 7] = self._dvdpitch(rover_state, opencv_pose_world, cam_pose_world)
#         H[1, 8] = self._dvdyaw(rover_state, opencv_pose_world, cam_pose_world)

#         return H

#     def meas(self, rover_state: np.ndarray) -> np.ndarray:
#         """
#         Measurement function for the EKF.
#         Input:
#             state - state vector of the EKF
#         Output:
#             z - measurement vector
#             H - measurement matrix
#             R - measurement noise covariance matrix
#         """

#         # get the lander origin pixel coordinates via projection to the image plane
#         lander_orig_pix, pix_unnorm = self.get_lander_origin_pix_from_proj(rover_state)
#         if  lander_orig_pix is None:
#             return None, None, None

#         # get the Jacobian
#         H = self.jac_reproj(rover_state, pix_unnorm)


#         # get the measurement noise covariance matrix
#         # TODO: tune based on segmentation results? like if not the entire lander is visible in the image etc.

#         return lander_orig_pix, H, self.R

In [ ]:
# # Ground truth trajectory data loading
# traj_data = json.load(open(f"{data_path}/data_log.json"))
# initial_pose = np.array(traj_data["initial_pose"])
# data_log_path = os.path.join(data_path, "data_log.json")
# data_log = json.load(open(data_log_path))

# world_pose = np.eye(4)
# lander_pose_world = np.array(data_log["lander_pose_world"])
# cam_pose_rover = get_cam_pose_rover(CAM_NAME)
# rover_pose_arr, rover_traj_times = rover_pose_world_traj(data_log)
# dt = np.diff(rover_traj_times)

# # plot reference frame poses in the world frame
# # fig = plot_reference_frames([world_pose, lander_pose_world, rover_pose_arr[0], rover_pose_arr[0] @ cam_pose_rover], ["world", "lander", "rover", "cam"])
# # fig.show()

In [ ]:
# def rover_state_from_pose(pose):
#     r, p, y = Rotation.from_matrix(pose[:3, :3]).as_euler("xyz")
#     return np.array([pose[0, 3], pose[1, 3], pose[2, 3], 0, 0, 0, r, p, y])

In [ ]:
# # extract IMU measurements
# imu_meas = []  # list of 6D IMU measurements (linear acceleration, angular velocity)
# for frame in data_log["frames"]:
#     imu_meas.append(np.array(frame["imu"], dtype=np.float64))

# # camera measurements (images)
# CAM_NAME = "Left"
# SEG_SCORE_THRESH = 0.4  # requires more testing
# model = LangSAM()
# img_folder = os.path.join(data_path, CAM_NAME)

# # Initialize filter
# v0 = np.zeros(3)
# pos0 = rover_pose_arr[0][:3, 3]
# rpy0 = Rotation.from_matrix(rover_pose_arr[0][:3, :3]).as_euler("xyz", degrees=False)
# x0 = np.hstack((pos0, v0, rpy0)).T

# init_r = 0.001
# init_v = 0.01
# init_angle = 0.001
# P0 = np.diag(
#     np.hstack(
#         (
#             np.ones(3) * init_r * init_r,
#             np.ones(3) * init_v * init_v,
#             np.ones(3) * init_angle * init_angle,
#         )
#     )
# )

# meas_model = MeasModelLanderSeg(
#     model, lander_pose_world, CAM_NAME, cams_geoms, seg_thresh=SEG_SCORE_THRESH, K=K
# )


# # Process noise
# Q_r = 0.00**2
# Q_v = 0.00**2
# Q_angle = 0.00005**2
# Q = np.diag(np.hstack((np.ones(3) * Q_r, np.ones(3) * Q_v, np.ones(3) * Q_angle)))

# # Measurement noise
# R = np.diag(np.array([1, 1]))  # 2D pixel measurements

# x_store = np.zeros((len(rover_pose_arr), 9))
# P_store = np.zeros((len(rover_pose_arr), 9, 9))

# ekf = EKF(x0, P0)

# for frame in data_log["frames"]:
#     frame_id = frame["frame"]

#     if frame_id == 0:
#         continue
#     else:
#         # print(f"Processing frame {frame_id}")
#         dt_f = dt[frame_id - 1]

#         # prediction
#         a_k = imu_meas[frame_id - 1][:3]
#         omega_k = imu_meas[frame_id - 1][3:]
#         dyn_func = lambda x: propagate_state(x, a_k, omega_k, dt_f, with_stm=True, use_numdiff=False)
#         ekf.predict(dyn_func, Q)

#         # extract lander origin pixel measurements from images
#         if os.path.exists(os.path.join(img_folder, f"{frame_id}.png")):
#             z = meas_model.get_lander_origin_pix_from_image(
#                 os.path.join(img_folder, f"{frame_id}.png")
#             )

#             if z is not None:
#                 ekf.update(z, meas_model.meas)
#             else:
#                 pass

#         else:
#             # if the image is not available, skip the update step
#             pass

#         # Compute error
#         error = ekf.x - rover_state_from_pose(rover_pose_arr[frame_id])

#         print("Frame: {:.2f} |  x_err: {:.3f} m | y_err: {:.3f} m | z_err: {:.3f} m | roll_err: {:.3f} deg | pitch_err: {:.3f} deg | yaw_err: {:.3f} deg".format(
#             frame_id,
#             error[0], error[1], error[2],
#             error[6] * 180 / np.pi, error[7] * 180 / np.pi, error[8] * 180 / np.pi
#         ))

#         x_store[frame_id] = ekf.x
#         P_store[frame_id] = ekf.P

# # extract lander origin pixel measurements from images
# # for frame in data_log["frames"][939:942]:
# #     frame_id = frame["frame"]
# #     print(f"Processing frame {frame_id}")
# #     img_name = os.path.join(img_folder, f"{frame_id}.png")

# #     if os.path.exists(img_name):
# #         lander_orig_pix_meas = meas_model.get_lander_origin_pix_from_image(img_name, display=True)

# #         # --------- test reproj ------------------
# #         print("Measured pixel coords: ", lander_orig_pix_meas)
# #         rover_pose_frame = np.array(frame["pose"])
# #         rover_rpy, rover_tvec = get_roteuler_tvec_from_poselist(rover_pose_frame)
# #         rover_state = np.concatenate([rover_tvec, np.zeros(3), rover_rpy])
# #         print(
# #             "Reproj Pixels: ",
# #             meas_model.get_lander_origin_pix_from_proj(rover_state, display=True),
# #         )
# #         # ---------------------------------------

In [ ]:
# def plot_results(gt_translations, gt_rotations_euler, x_store, P_store, title=""):
#     fig, axes = plt.subplots(3, 3, figsize=(15, 10))

#     n_sim = x_store.shape[0]

#     # positions -------------------------------
#     pos_labels = ["X", "Y", "Z", "Vx", "Vy", "Vz"]

#     for i in range(6):
#         ax = axes.flatten()[i]

#         # ax.plot(gt_translations[:n_sim, i], label="True")
#         # ax.plot(x_store[:, i], label="IMU", alpha=0.5)
#         # # plot 3-sigma bounds
#         # ax.fill_between(
#         #     np.arange(n_sim),
#         #     x_store[:, i] - 2 * np.sqrt(P_store[:, i, i]),
#         #     x_store[:, i] + 2 * np.sqrt(P_store[:, i, i]),
#         #     alpha=0.2,
#         #     label="2-sigma bounds",
#         # )

#         ax.plot(gt_translations[:n_sim, i] - x_store[:, i], label="True")
#         # plot 3-sigma bounds
#         ax.fill_between(
#             np.arange(n_sim),
#             - 2 * np.sqrt(P_store[:, i, i]),
#             + 2 * np.sqrt(P_store[:, i, i]),
#             alpha=0.2,
#             label="2-sigma bound"
#         )

#         ax.legend()
#         ax.grid()
#         ax.set_xlabel("Timestep (0.1 s)")
#         ax.set_ylabel(f"{pos_labels[i]}")

#     # angles -----------------------------
#     RAD2DEG = 180 / np.pi
#     pos_labels = ["Err Roll (deg)", "Err Pitch (deg)", "Err Yaw (deg)"]
#     for i in range(3):
#         ax = axes.flatten()[i + 6]
#         ax.plot((gt_rotations_euler[:n_sim, i] - x_store[:, 6+i]) * RAD2DEG, label="True")
#         # plot 3-sigma bounds
#         ax.fill_between(
#             np.arange(n_sim),
#             - 2 * np.sqrt(P_store[:, 6+i, 6+i]) * RAD2DEG,
#             + 2 * np.sqrt(P_store[:, 6+i, 6+i]) * RAD2DEG,
#             alpha=0.2,
#             label="2-sigma bounds"
#         )
#         ax.legend()
#         ax.grid()
#         ax.set_xlabel("Timestep (0.1 s)")
#         ax.set_ylabel(f"{pos_labels[i]}")

#     plt.suptitle(title)
#     plt.tight_layout()
#     plt.show()

# Using fiducials


In [ ]:
# # known fiducial corner positions
# fiducial_group_centers = json.load(
#     open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json"))
# )["lander"]["fiducials"]
# centers_group_a = fiducial_group_centers["a"]
# centers_group_b = fiducial_group_centers["b"]
# centers_group_c = fiducial_group_centers["c"]
# centers_group_d = fiducial_group_centers["d"]

# FIDUCIAL_NAMES = ["top left", "top right", "lower right", "lower left"]


# def fiducial_centers_array(centers: dict) -> np.ndarray:
#     center_pts = []
#     for name in FIDUCIAL_NAMES:
#         center_pts.append(np.array([centers[name]["x"], centers[name]["y"], centers[name]["z"]]))
#     return np.array(center_pts)

## Segementation


In [ ]:
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/lander_closeups/")
# data_log = json.load(open(os.path.join(data_path, "data_log.json")))
# CAM_NAME = "FrontLeft"


# # ----- lander closeups -----------------------------------------------
# # true examples
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/31.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/65.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/97.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/147.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/181.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/265.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/353.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/369.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/411.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/457.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/493.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/545.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1199.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1269.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1665.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1705.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1751.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/2043.png

# # difficult examples
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/799.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/867.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1111.png

# # false examples
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/617.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/709.png
# # /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/763.png

# i = [
#     # 31,
#     # 65,
#     # 97,
#     # 147,
#     # 181,
#     # 265,
#     353,
#     #     369,
#     #     411,
#     #     457,
#     #     493,
#     #     545,
#     #     1199,
#     #     1269,
#     #     1665,
#     #     1705,
#     #     1751,
#     #     2043,
#     #     799,
#     #     867,
#     #     1111,
#     #     617,
#     #     709,
#     #     763,
# ]

In [ ]:
# # Check reference frame alignments
# FRAME_ID = 353

# opencv_pose_cam = np.eye(4)
# opencv_pose_cam[:3, :3] = np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]]).T

# world_pose = np.eye(4)
# lander_pose_world = np.array(data_log["lander_pose_world"])
# rover_poses_world, rover_poses_timestamps = rover_pose_world_traj(data_log)
# cam_pose_rover = get_cam_pose_rover(CAM_NAME, cams_geoms)
# cam_pose_world_init = rover_poses_world[FRAME_ID] @ cam_pose_rover
# opencv_pose_world = cam_pose_world_init @ opencv_pose_cam

# fig = plot_reference_frames(
#     [
#         world_pose,
#         lander_pose_world,
#         rover_poses_world[FRAME_ID],
#         cam_pose_world_init,
#         opencv_pose_world,
#     ],
#     ["world", "lander", "rover", "cam", "opencv"],
# )
# fig.show()


# # ## FRAME TESTS
# # cam_pose_world = np.eye(4)
# # cam_pose_world[0, 3] = -1
# # test_pt = np.array([1, 0, 0, 1])

# # opencv_pose_world = cam_pose_world @ opencv_pose_cam
# # print(opencv_pose_world)

# # test_cam_pose_world_passive = invert_transform(cam_pose_world)
# # test_opencv_pose_world_passive = invert_transform(opencv_pose_world)
# # print(test_cam_pose_world_passive @ test_pt)
# # print(test_opencv_pose_world_passive @ test_pt)

In [ ]:
# # Segment fiducials
# model = LangSAM()
# text_prompt = "square."
# SEG_SCORE_THRESH = 0.4  # requires more testing

# for ind in i:
#     # segment the image
#     I1 = cv.imread(os.path.join(data_path, CAM_NAME, f"{ind}.png"), cv.IMREAD_GRAYSCALE)
#     plt.imshow(I1, cmap="gray")
#     plt.show()

#     image_seg_in = Image.open(os.path.join(data_path, CAM_NAME, f"{ind}.png")).convert("RGB")
#     results = model.predict([image_seg_in], [text_prompt])
#     full_mask = np.zeros_like(image_seg_in).copy()

#     square_center_pix = []
#     for j, mask in enumerate(results[0]["masks"]):
#         if results[0]["scores"][j] < SEG_SCORE_THRESH:
#             continue
#         full_mask[mask.astype(bool)] = 255

#         # extract the square center pixel
#         contour_mask = mask.astype(np.uint8)
#         contours, _ = cv.findContours(contour_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#         if contours:
#             largest_contour = max(contours, key=cv.contourArea)
#             rect = cv.minAreaRect(largest_contour)
#             box = cv.boxPoints(rect)
#             box = np.intp(box)
#             mid_point = np.mean(box, axis=0)
#             square_center_pix.append(mid_point)

#             # show on image
#             cv.drawContours(full_mask, [box], 0, (0, 0, 255), 2)
#             cv.circle(full_mask, tuple(mid_point.astype(int)), 10, (0, 0, 255), -1)

#     # test reprojection with given robot pose
#     rover_pose_world = rover_poses_world[ind, :, :]
#     cam_pose_world = get_cam_pose_rover(CAM_NAME, cams_geoms) @ rover_pose_world
#     opencv_pose_world = cam_pose_world @ opencv_pose_cam
#     opencv_pose_world_passive = invert_transform_mat(opencv_pose_world)

#     fid_center_m = (
#         lander_pose_world @ np.hstack((fiducial_centers_array(centers_group_c), np.ones((4, 1)))).T
#     )
#     fid_center_pix_sort = np.array(sorted(square_center_pix, key=lambda x: x[1]))

#     for j, fc in enumerate(fid_center_m.T):
#         coord_cam = opencv_pose_world_passive @ fc.T
#         print("Coord in opencv frame: ", coord_cam)
#         reproj_pix = K @ np.hstack((np.eye(3), np.zeros((3, 1)))) @ coord_cam
#         reproj_pix = reproj_pix[:2] / reproj_pix[2]
#         print("Reproj pix: ", reproj_pix)
#         cv.circle(full_mask, tuple(reproj_pix.astype(int)), 10, (255, 0, 0), -1)

#     plt.imshow(full_mask, cmap="gray")
#     plt.show()

## AprilTag


In [ ]:
from lac.localization.ekf import create_Q, get_pose_measurement_tag
from lac import params
from lac.util import pose_to_pos_rpy

# load data
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/localization_preset_0_seed_4_10hz/")
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/LocalizationAgent_spiral_norocks/")
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/MappingAgent_map1_preset0_run2/")
data_path = os.path.expanduser(
    "~/LunarAutonomyChallenge/output/LocalizationAgent/map1_preset0_maxdist_fiducials/"
)
json_data = json.load(open(os.path.join(data_path, "data_log.json")))
initial_pose, lander_pose, pose_list, imu_data, cam_config = load_data(data_path)

# initialize the apriltag detector
fid_localizer = FiducialLocalizer(cam_config)

### Run EKF


In [ ]:
# Define the ground truth trajectory ------------------------------
gt_translations = np.zeros((len(pose_list), 3))  # x, y, z
gt_rotations_euler = np.zeros((len(pose_list), 3))  # roll, pitch, yaw

for i in range(0, len(pose_list)):
    pos_i, rpy_i = pose_to_pos_rpy(pose_list[i])
    gt_translations[i, :3] = pos_i
    gt_rotations_euler[i] = rpy_i

# true state storage
n_sim = len(pose_list)
true_states = np.zeros((n_sim, 9))
true_states[0] = np.hstack(
    (
        gt_translations[0, :3],
        (gt_translations[1, :3] - gt_translations[0, :3]) / params.DT,
        gt_rotations_euler[0],
    )
)
for i in range(1, n_sim):
    true_states[i] = np.hstack(
        (
            gt_translations[i, :3],
            (gt_translations[i, :3] - gt_translations[i - 1, :3]) / params.DT,
            gt_rotations_euler[i],
        )
    )

# display the ground truth trajectory
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=gt_translations[:, 0], y=gt_translations[:, 1], mode="lines", name="X"))
# fig.show()

In [ ]:
# Generate true measurements ----------------------------------------

# get image names to determine the measurement step size
# cameras = ["FrontLeft", "Right"]
CAM_NAME = "Right"  # TODO: use both cameras
img_names = os.listdir(os.path.join(data_path, CAM_NAME))
img_inds = [int(img_name.split(".")[0]) for img_name in img_names]
img_inds.sort()
meas_interval = img_inds[1] - img_inds[0]
print("meas_inv: ", meas_interval)


true_meas = {}
n_meas = {}

for i in range(1, n_sim):
    if i in img_inds:
        # measurement ----------------------------------------
        img_name = os.path.join(data_path, CAM_NAME, f"{i}.png")
        img = cv.imread(img_name, cv.IMREAD_GRAYSCALE)
        meas_pose, _ = fid_localizer.estimate_rover_pose(img, CAM_NAME, lander_pose)
        true_meas_list = [
            np.concatenate(pose_to_pos_rpy(pose)) for pose in meas_pose.values()
        ]
        true_meas_array = np.array(true_meas_list).flatten()
        true_meas[i] = true_meas_array
        n_meas[i] = len(meas_pose)

In [ ]:
def get_pose_measurement_tag_adapt(x, nmeas, yaw_rate, dist_to_lander):
    """
    Generate a measurement vector and the corresponding Jacobian and covariance matrix

    Args:
    x: state vector (9x1)  [x, y, z, vx, vy, vz, roll, pitch, yaw]
    nmeas: number of measurements (detected tags)
    """

    H = np.zeros((0, 9))
    Rdiag = np.zeros((0))

    meas = np.zeros(6 * nmeas)

    for j in range(nmeas):
        meas[6 * j : 6 * j + 3] = x[:3]
        meas[6 * j + 3 : 6 * j + 6] = x[6:]

        # Jacobian of the measurement function
        Htmp = np.zeros((6, 9))
        Htmp[:3, :3] = np.eye(3)
        Htmp[3:, 6:] = np.eye(3)

        H = np.vstack([H, Htmp])

        # TODO: move these params
        std_x = 0.25 * (1 + dist_to_lander / 20 + yaw_rate / 2)
        std_y = 0.25 * (1 + dist_to_lander / 20 + yaw_rate / 2)
        std_z = 0.25 * (1 + dist_to_lander / 20 + yaw_rate / 2)
        std_roll = 0.05
        std_pitch = 0.05
        std_yaw = 0.2

        Rtmp = np.array(
            [std_x**2, std_y**2, std_z**2, std_roll**2, std_pitch**2, std_yaw**2]
        )
        Rdiag = np.concatenate([Rdiag, Rtmp])

    R = np.diag(Rdiag)

    return meas, H, R

In [ ]:
# Initialize EKF ---------------------------------------------------
R_0 = pose_list[0][:3, :3]

imu_rotations = []
imu_rotations.append(R_0)

init_pos, init_rpy = pose_to_pos_rpy(pose_list[0])
v0 = np.zeros(3)
init_state = np.hstack((init_pos, v0, init_rpy)).T
P0 = params.EKF_P0

# Process noise
Q_EKF = create_Q(params.DT, params.EKF_Q_SIGMA_A, params.EKF_Q_SIGMA_ANGLE)
# Q_EKF = create_Q(params.DT, 0.05, 0.0005)
ekf = EKF(init_state, P0, store=True)  # EKF with IMU + Measurement

x_store = np.zeros((n_sim, 9))
P_store = np.zeros((n_sim, 9, 9))
x_store[0] = ekf.x
P_store[0] = ekf.P

time = 0
times = [0]
meas_tidx = []


# Run EKF ----------------------------------------------------------
for i in range(1, n_sim):
    time += params.DT
    times.append(time)

    a_k = imu_data[i - 1][:3]  # imu data starts with step 1 at index 0
    omega_k = imu_data[i - 1][3:]

    def dyn_func(x):
        return propagate_state(
            x, a_k, omega_k, params.DT, with_stm=True, use_numdiff=False
        )

    # ekfi.predict(i, dyn_func, Q_EKF)
    ekf.predict(i, dyn_func, Q_EKF)

    # sample image every 1 sec
    if i in img_inds:
        # measurement ----------------------------------------
        if n_meas[i] > 0:

            def meas_func(x):
                return get_pose_measurement_tag(x, n_meas[i])

            est_lander_dist = np.linalg.norm(ekf.x[:3] - lander_pose[:3, 3])
            # def meas_func(x):
            # return get_pose_measurement_tag_adapt(x, n_meas[i], omega_k[2], est_lander_dist)

            ekf.update(i, true_meas[i], meas_func)

        # Compute error
        error = ekf.x - true_states[i]

        print(
            "Time: {:.2f} s |  x_err: {:.3f} m | y_err: {:.3f} m | z_err: {:.3f} m | roll_err: {:.3f} deg | pitch_err: {:.3f} deg | yaw_err: {:.3f} deg".format(
                time,
                error[0],
                error[1],
                error[2],
                error[6] * 180 / np.pi,
                error[7] * 180 / np.pi,
                error[8] * 180 / np.pi,
            )
        )

        meas_tidx.append(i)

        if i % params.EKF_SMOOTHING_INTERVAL == 0:
            ekf.smooth()

    x_store[i] = ekf.x
    P_store[i] = ekf.P

# smooth
rms_error = np.sqrt(np.mean((x_store[:, :3] - true_states[:, :3]) ** 2, axis=0))
print(f"EKF | RMS Position Error: {rms_error}")

# run smoother
ekf.smooth()

# run iterations
n_iter = 3
for itr in range(n_iter):
    # Compute the rms position error
    res = ekf.get_results()
    rms_err = np.sqrt(
        np.mean((res["xhat_smooth"][:, :3] - true_states[:, :3]) ** 2, axis=0)
    )

    print(f"Iteration {itr + 1} | RMS Position Error: {rms_err}")

    ekf = EKF(ekf.xhat_store_smooth[0], P0, store=True)  # EKF with IMU + Measurement

    for i in range(1, n_sim):
        a_k = imu_data[i - 1][:3]
        omega_k = imu_data[i - 1][3:]

        def dyn_func(x):
            return propagate_state(
                x, a_k, omega_k, params.DT, with_stm=True, use_numdiff=False
            )

        ekf.predict(i, dyn_func, Q_EKF)

        # sample image every 1 sec
        if i in img_inds:
            # measurement ----------------------------------------
            if n_meas[i] > 0:

                def meas_func(x):
                    return get_pose_measurement_tag(x, n_meas[i])

                est_lander_dist = np.linalg.norm(ekf.x[:3] - lander_pose[:3, 3])
                # def meas_func(x):
                #     return get_pose_measurement_tag_adapt(x, n_meas[i], omega_k[2], est_lander_dist)

                ekf.update(i, true_meas[i], meas_func)

    # run smoother
    ekf.smooth()

res = ekf.get_results()

data_ekf = {"times": times, "x_err": x_store - true_states, "P_store": P_store}
data_smooth = {
    "times": times,
    "x_err": res["xhat_smooth"] - true_states,
    "P_store": res["Phat_smooth"],
}

rms_pos = {}
rms_angle = {}

rms_pos["meas"] = np.sqrt(np.mean(data_ekf["x_err"][:, :3] ** 2, axis=0))
rms_pos["smooth"] = np.sqrt(np.mean(data_smooth["x_err"][:, :3] ** 2, axis=0))

rms_angle["meas"] = np.sqrt(np.mean(data_ekf["x_err"][:, 6:] ** 2, axis=0))
rms_angle["smooth"] = np.sqrt(np.mean(data_smooth["x_err"][:, 6:] ** 2, axis=0))

### Postprocessing


In [ ]:
def plot_results(data, title=""):
    times = data["times"]
    xerr = data["x_err"]
    P_store = data["P_store"]

    fig, axes = plt.subplots(3, 3, figsize=(15, 10))

    n_sim = xerr.shape[0]

    # positions -------------------------------
    pos_labels = ["X", "Y", "Z", "Vx", "Vy", "Vz"]

    for i in range(6):
        ax = axes.flatten()[i]

        ax.plot(times, xerr[:, i], label="True")
        # plot 3-sigma bounds
        ax.fill_between(
            times,
            -3 * np.sqrt(P_store[:, i, i]),
            +3 * np.sqrt(P_store[:, i, i]),
            alpha=0.2,
            label="3-sigma bound",
        )

        ax.legend()
        ax.grid()
        ax.set_xlabel("Time")
        ax.set_ylabel(f"{pos_labels[i]}")

    # angles -----------------------------
    RAD2DEG = 180 / np.pi
    pos_labels = ["Err Roll (deg)", "Err Pitch (deg)", "Err Yaw (deg)"]
    for i in range(3):
        ax = axes.flatten()[i + 6]
        ax.plot(times, xerr[:, i + 6] * RAD2DEG, label="True")
        # plot 3-sigma bounds
        ax.fill_between(
            times,
            -3 * np.sqrt(P_store[:, 6 + i, 6 + i]) * RAD2DEG,
            +3 * np.sqrt(P_store[:, 6 + i, 6 + i]) * RAD2DEG,
            alpha=0.2,
            label="3-sigma bounds",
        )
        ax.legend()
        ax.grid()
        ax.set_xlabel("Time")
        ax.set_ylabel(f"{pos_labels[i]}")
        ax.set_ylim(-1, 1)

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

In [ ]:
print("Measurement RMS Pos: ", rms_pos["meas"])
print("Measurement RMS Angle: ", rms_angle["meas"])

plot_results(data_ekf, title="IMU + Measurement")

In [ ]:
print("Smoothed RMS Pos: ", rms_pos["smooth"])
print("Smoothed RMS Angle: ", rms_angle["smooth"])

plot_results(data_smooth, title="Smoothed")

In [ ]:
from lac.utils.plotting import plot_path_3d, plot_3d_points

lander_position = lander_pose[:3, 3]

fig = plot_path_3d(
    gt_translations,
    color="blue",
    name="Ground truth",
)
fig = plot_3d_points(
    lander_position[None, :], fig=fig, color="red", markersize=3, name="Lander Origin"
)
# fig = plot_path_3d(
#     x_store_imu[:, :3],
#     fig=fig,
#     color="orange",
#     name="EKF with IMU",
# )
fig = plot_path_3d(
    x_store[:, :3],
    fig=fig,
    color="green",
    name="EKF with Measurements",
)
fig = plot_path_3d(
    res["xhat_smooth"][:, :3],
    fig=fig,
    color="purple",
    name="Smoothed",
)

if "ekf_result.npz" in os.listdir(data_path):
    ekf_results = np.load(os.path.join(data_path, "ekf_result.npz"))
    ekf_x = ekf_results["xhat_smooth"]
    fig = plot_path_3d(
        ekf_x[:, :3],
        fig=fig,
        color="orange",
        name="EKF Online",
    )

true_meas_array = np.concatenate([v for v in true_meas.values() if v.size > 0], axis=0)
if true_meas_array.ndim == 1:
    true_meas_array = true_meas_array.reshape(-1, 6)
fig = plot_3d_points(
    true_meas_array[:, :3], fig=fig, color="red", markersize=3, name="True Measurements"
)


fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()

In [ ]:
true_vel = []
smoothed_vel = []
for i in range(len(true_states)):
    true_vel.append(np.linalg.norm(true_states[i, 3:6]))
    smoothed_vel.append(np.linalg.norm(res["xhat_smooth"][i, 3:6]))


fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.arange(0, len(true_states)), y=true_vel, mode="lines", name="True")
)
fig.add_trace(
    go.Scatter(
        x=np.arange(0, len(true_states)), y=smoothed_vel, mode="lines", name="Smoothed"
    )
)
fig.update_layout(hovermode="x unified")